# MAE345 Midterm: Theory Q1 Calculations
## Alkin Kaz (due 11/9/2022)

## (Q1.b) Linearizing the Dynamics of the Furuta Pendulum

We'll follow similar calculations to what we did in Lab1.

In [1]:
import sympy as sp
import numpy as np
from sympy.physics.vector import dynamicsymbols as dynamicsymbols

In [2]:
# constants and time
g, I_0, L_0, m_1, l_1, J_1, t = sp.symbols("g I_0 L_0 m_1 l_1 J_1 t")
numeric_vals = [(g, 9.81), (I_0, 1), (L_0, 1), (m_1, 1), (l_1, 1), (J_1, 1)]

# control input
tau = sp.symbols("tau")

# state variables
theta_0, theta_1 = dynamicsymbols("theta_0 theta_1")
theta_0_dot = sp.diff(theta_0, t)
theta_1_dot = sp.diff(theta_1, t)
q_bar_dot = sp.Matrix([theta_0_dot, theta_1_dot])

In [3]:
# derived expressions: F, g(q_bar), C(q_bar, q_bar_dot), M(q_bar)
F = sp.Matrix([tau, 0])

g_func = sp.Matrix([
    0,
    - m_1 * l_1 * g * sp.sin(theta_1)
])

C_11 = m_1 * l_1**2 * theta_1_dot * sp.sin(2*theta_1) / 2
C_12 = m_1 * l_1**2 * theta_0_dot * sp.sin(2*theta_1) / 2 \
    - m_1 * l_1 * L_0 * theta_1_dot * sp.sin(theta_1)
C_21 = - m_1 * l_1**2 * theta_0_dot * sp.sin(2*theta_1) / 2
C_22 = 0

C = sp.Matrix([
    (C_11, C_12),
    (C_21, C_22)
])

M_11 = I_0 + m_1*(L_0**2 + l_1**2 * (sp.sin(theta_1))**2)
M_12 = m_1 * l_1 * L_0 * sp.cos(theta_1)
M_21 = m_1 * l_1 * L_0 * sp.cos(theta_1)
M_22 = J_1 + m_1 * l_1**2

M = sp.Matrix([
    (M_11, M_12),
    (M_21, M_22)
])

In [4]:
# dynamics / EoM of the system:

eq_second_order = M.inv() @ (F - C @ q_bar_dot - g_func)

state = sp.Matrix([theta_0, theta_1, theta_0_dot, theta_1_dot])
input = sp.Matrix([tau])

dynamics = sp.Matrix([
    theta_0_dot,
    theta_1_dot,
    eq_second_order[0],
    eq_second_order[1]
])

In [5]:
# linearization
A = dynamics.jacobian(state)
B = dynamics.jacobian(input)

In [6]:
# plugging in the reference point

# interestingly, the order matters. once theta_0 is
# subbed, the connection to theta_0_dot is lost
ref_state = [
    (theta_0_dot, 0),
    (theta_1_dot, 0),
    (theta_0, 0),
    (theta_1, 0)
]
ref_input = [(tau, 0)]

A_symbolic = A.subs(ref_state).subs(ref_input)
B_symbolic = B.subs(ref_state).subs(ref_input)

# check if the reference point is actually a fixed point
dynamics.subs(ref_state).subs(ref_input)

Matrix([
[0],
[0],
[0],
[0]])

In [7]:
# plugging in the numeric values
A_arr = np.array(A_symbolic.subs(numeric_vals)).astype(float)
B_arr = np.array(B_symbolic.subs(numeric_vals)).astype(float)

In [8]:
print(A_arr)

[[ 0.    0.    1.    0.  ]
 [ 0.    0.    0.    1.  ]
 [ 0.   -3.27  0.    0.  ]
 [ 0.    6.54  0.    0.  ]]


In [9]:
print(B_arr)

[[ 0.        ]
 [ 0.        ]
 [ 0.66666667]
 [-0.33333333]]


In [10]:
# checking the symbolic A and B
A_symbolic

Matrix([
[0,                                                                        0, 1, 0],
[0,                                                                        0, 0, 1],
[0,         -L_0*g*l_1**2*m_1**2/(I_0*J_1 + I_0*l_1**2*m_1 + J_1*L_0**2*m_1), 0, 0],
[0, g*l_1*m_1*(I_0 + L_0**2*m_1)/(I_0*J_1 + I_0*l_1**2*m_1 + J_1*L_0**2*m_1), 0, 0]])

In [11]:
B_symbolic

Matrix([
[                                                             0],
[                                                             0],
[(J_1 + l_1**2*m_1)/(I_0*J_1 + I_0*l_1**2*m_1 + J_1*L_0**2*m_1)],
[      -L_0*l_1*m_1/(I_0*J_1 + I_0*l_1**2*m_1 + J_1*L_0**2*m_1)]])

## (Q1.c) LQR of the Furuta Pendulum

We will follow what we did in Lab2.

In [12]:
from scipy.linalg import solve_continuous_are

def lqr(A: np.ndarray, B: np.ndarray, Q: np.ndarray, R: np.ndarray) -> np.ndarray:
    S = solve_continuous_are(A, B, Q, R)
    K = -np.linalg.inv(R) @ B.T @ S
    return K

In [13]:
Q = np.eye(len(A_arr[0]))
R = np.eye(len(B_arr[0]))
K = lqr(np.array(A_arr), np.array(B_arr), Q, R)
print(K)

[[ 1.         60.31467838  3.0490278  24.93652874]]
